In [1]:
import gym
import myrl
from myrl.buffers import ReplayBuffer, PrioritizedReplayBuffer
from myrl.environments import Envs
from myrl.value_functions import ValueFunctionMLP, polyak
from myrl.policies import LinearPolicy, RandomPolicy
from myrl.visualizer import showit
from myrl.utils import normal_noise, ExperimentWriter
import torch    
import copy
from collections import deque
import numpy as np


z = 0
envname = 'LunarLanderContinuous-v2'
env = gym.make(envname)
envs = Envs(envname, 3)
random_policy = RandomPolicy(env).act
obsdim = env.observation_space.shape[0]
adim = env.action_space.shape[0]
wll = ExperimentWriter('tb/lunarnovi')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [13]:
rbuff = ReplayBuffer(max_len=50000, device=torch.device('cpu'))

maxer = LinearPolicy(obsdim, 32, adim)
q1 = ValueFunctionMLP(obsdim+adim, 64, None, 1)
q2 = ValueFunctionMLP(obsdim+adim, 64, None, 1)
tq1 = copy.deepcopy(q1)
tq2 = copy.deepcopy(q2)
opt = torch.optim.Adam(list(q1.parameters())+list(q2.parameters()), lr=1e-1)
mopt = torch.optim.Adam(maxer.parameters(), lr=1e-0)

In [15]:
wll.new()
writer = wll.writer
for i in range(1):
    eps = 10000
    gamma = 0.992
    bsize = 512
    warmup = 5000
    maxeract = lambda obs: maxer.act(obs, std=0.3)

    for ep in range(eps):

        for i in range(1):
            pi = random_policy if len(rbuff)<warmup else maxeract
            oldobs, a, r, obs, d, _, _, _ = envs.rollout(pi,debug=0)
            rbuff.add(oldobs, a, r, obs, d)
        if len(rbuff) < bsize*3:
            print(len(rbuff))
            continue  

        for put in range(4):
            oldobs, a, r, obs, d = rbuff.get(bsize)

            for optstep in range(2):
                amax = maxer(obs).detach()
                amax += normal_noise(amax, 0.08)
                in1 = torch.cat((obs, amax), dim=-1)
                in2 = torch.cat((oldobs, a), dim=-1)
                target = r + gamma*(1-d*0)*torch.min(tq1(in1), tq2(in1))[0]
                td_error = ((target - q1(in2))**2).mean() + ((target - q2(in2))**2)
                td = td_error.mean()
                opt.zero_grad()
                td.backward()
                opt.step()
            
            if not put%2 or 1:
                for optstep in range(4):
                    amax = maxer(obs)
                    in1 = torch.cat((obs, amax), dim=-1)
                    maxer_loss = -q1(in1).mean()
                    mopt.zero_grad()
                    maxer_loss.backward()
                    mopt.step()

            polyak(q1, tq1, 1-1/70)
            polyak(q2, tq2, 1-1/70)

        writer.add_scalar("multi/maxer_loss", maxer_loss.item(), ep)
        writer.add_scalar("multi/td_q", td.item(), ep)
        writer.add_scalar("multi/rew", r.mean().item(), ep)

        if ep%20==0:
            print(ep, maxer_loss.item(), r.mean().item(), td.item(), len(rbuff))
        if ep%200==0:
            showit(env, maxer.act)
            env.close()


torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
0 102.77114868164062 -1.7318589687347412 156.7296142578125 50000
63 /2000torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])
torch.Size([512, 1])


KeyboardInterrupt: 

In [17]:
torch.tesnor

(2, 3, 2, 3)